## From npy to mid (one bar) chord sequence

In [7]:
import numpy as np
from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
import os
import shutil
import pretty_midi
import librosa.display
import matplotlib.pyplot as plt

for i in range(1):
    print(i)
    gen = np.load('../exps/nowbar_hybrid/gen/gen.npy')
    tt = np.load('../data/chord_roll/val/x_bar_chroma_Amazing_Grace.npy')
    #tt = np.load('../data/chord_roll/val/x_bar_chroma_8bar_vae.npy')
    madmom_cho = np.load('../data/chord_roll/val/y_bar_chroma_Amazing_Grace.npy')
    #madmom_cho = np.load('../data/chord_roll/val/y_bar_chroma_8bar_vae.npy')
    print(np.shape(madmom_cho))
    
    temp_tt_mel = tt[:64,:,:,:1]
    temp_tt_cho = tt[:64,:,:,1:2]
    temp_madmom_cho = madmom_cho[:64,:,:,:]
    ##temp_tt_mel = tt[64:128,:,:,:1]
    ##temp_tt_cho = tt[64:128,:,:,1:2]
    ##temp_madmom_cho = madmom_cho[64:128,:,:,:]
    
    print(np.shape(gen))
    print(np.shape(temp_tt_cho))
    
    # move a direction
    ##source = '../musegan/interpolation/gen_4dbar/interp_%s'%i +'/sample/'
    ##destination = '../musegan/interpolation/gen_4dbar/interp/pic_%s'%i+'/'
    ##if not os.path.exists(destination):
    ##    os.makedirs(destination)
    ##os.rename(source, destination)

    gen_pr = np.zeros((64, 48, 128, 8), dtype=bool)
    #print(np.shape(gen_pr[:,:,24:108,:]))
    gen_pr[:,:,24:108,:5] = gen
    gen_pr[:,:,24:108,5:6] = temp_tt_mel
    gen_pr[:,:,24:108,6:7] = temp_tt_cho
    gen_pr[:,:,24:108,7:8] = temp_madmom_cho
    #print(np.shape(gen_pr[:,:,24:108,:]))
    gen_pr_float = gen_pr.astype(float)*1.0
    #-------------------------------#
    # deal with overfragmented note #
    #-------------------------------#
    gen_32th = np.zeros((64,16,128,8), dtype=float) # (interp_idx, time_idx, pitch_idx, track_idx)
    
    for idx_song in range(64):
        for idx in range(8):
            for pitch in range(128):
                for time in range(16):
                    if ((time%16) == 15) & (idx == 6 or 7):
                        gen_32th[idx_song,time,pitch,idx]=0.0
                        continue
                    else:
                        pass
                    cal = (gen_pr_float[idx_song,3*time,pitch,idx]+ gen_pr_float[idx_song,3*time+1,pitch,idx]+ gen_pr_float[idx_song,3*time+2,pitch,idx])/3
                    #print(cal)
                    if idx == 1:
                        if cal > 0:
                            gen_32th[idx_song,time,pitch,idx]=1.0
                        else:
                            gen_32th[idx_song,time,pitch,idx]=0.0
                    elif idx == 2 or idx == 3:
                        if cal > 0.8:
                            gen_32th[idx_song,time,pitch,idx]=1.0
                        else:
                            gen_32th[idx_song,time,pitch,idx]=0.0
                    else:
                        if cal > 0.6:
                            gen_32th[idx_song,time,pitch,idx]=1.0
                        else:
                            gen_32th[idx_song,time,pitch,idx]=0.0
    
    #------------------#
    # save numpy files #
    #------------------#
    ##directory = '../musegan/interpolation/gen_4dbar/interp/all_npy/'
    ##if not os.path.exists(directory):
    ##    os.makedirs(directory)
    ##file_o = directory + 'all_%s'%i + '.npy'
    ##np.save(file_o, gen_32th)
    
    
    #------------------#
    # write midi files #
    #------------------#
        
    # All
    for idx in range(len(gen_32th[:,0,0,0])):
        # Create a `pypianoroll.Track` instance
        # program can be choosed to apply various instruments
        track_bass = Track(pianoroll=gen_32th[idx,:,:,0], program=33, is_drum=False,
                      name= 'bass')
        track_drum = Track(pianoroll=gen_32th[idx,:,:,1], program=0, is_drum=True,
                      name= 'drum')
        track_guitar = Track(pianoroll=gen_32th[idx,:,:,2], program=25, is_drum=False,
                      name= 'guitar')
        track_piano = Track(pianoroll=gen_32th[idx,:,:,3], program=0, is_drum=False,
                      name= 'piano')
        track_string = Track(pianoroll=gen_32th[idx,:,:,4], program=41, is_drum=False,
                      name= 'string')
        track_tt_mel = Track(pianoroll=gen_32th[idx,:,:,5], program=0, is_drum=False,
                      name= 'tt_mel')
        track_tt_cho = Track(pianoroll=gen_32th[idx,:,:,6], program=0, is_drum=False,
                      name= 'tt_cho')
        track_madmom_cho = Track(pianoroll=gen_32th[idx,:,:,7], program=0, is_drum=False,
                      name= 'madmom_cho')
    
        # Create a `pypianoroll.Multitrack` instance
        multitrack = Multitrack(tracks=[track_bass, track_drum, track_guitar, track_piano, track_string, track_tt_mel, track_tt_cho, track_madmom_cho], tempo=100.0, beat_resolution=4)
        
        # Plot the piano-roll
        ##fig, ax = multitrack.plot()
        ##plt.show()
        
        # Write the `pypianoroll.Multitrack` instance to a MIDI file
        directory = '../exps/nowbar_hybrid/gen_4dbar/all/'
        if not os.path.exists(directory):
            os.makedirs(directory)
        path_o = directory + 'all_%s'%i+'_%s'%idx + '.mid'
        multitrack.write(path_o)
        
    gen_32th = np.reshape(gen_32th,[-1,128,8])
    
    # 64 samples
    track_bass = Track(pianoroll=gen_32th[:,:,0], program=33, is_drum=False,
                      name= 'bass')
    track_drum = Track(pianoroll=gen_32th[:,:,1], program=0, is_drum=True,
                      name= 'drum')
    track_guitar = Track(pianoroll=gen_32th[:,:,2], program=25, is_drum=False,
                      name= 'guitar')
    track_piano = Track(pianoroll=gen_32th[:,:,3], program=0, is_drum=False,
                      name= 'piano')
    track_string = Track(pianoroll=gen_32th[:,:,4], program=41, is_drum=False,
                      name= 'string')
    track_tt_mel = Track(pianoroll=gen_32th[:,:,5], program=0, is_drum=False,
                      name= 'tt_mel')
    track_tt_cho = Track(pianoroll=gen_32th[:,:,6], program=0, is_drum=False,
                      name= 'tt_cho')
    track_madmom_cho = Track(pianoroll=gen_32th[:,:,7], program=0, is_drum=False,
                      name= 'madmom_cho')
    
    
    # Create a `pypianoroll.Multitrack` instance
    multitrack = Multitrack(tracks=[track_bass, track_drum, track_guitar, track_piano, track_string, track_tt_mel, track_tt_cho, track_madmom_cho], tempo=100.0, beat_resolution=4)
        
    # Plot the piano-roll
    ##fig, ax = multitrack.plot()
    ##plt.show()
        
    # Write the `pypianoroll.Multitrack` instance to a MIDI file
    directory = '../exps/nowbar_hybrid/gen_4dbar/all/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    path_o = directory + 'all_%s'%i + '.mid'
    multitrack.write(path_o)    

0
(64, 48, 84, 1)
(64, 48, 84, 5)
(64, 48, 84, 1)


## From mid to preformance mid

In [8]:
import os
import pretty_midi
import numpy as np
##directory = '../musegan/exps/NowBarRNN_hybrid/gen_4dbar/all_dyn/'
directory = '../exps/nowbar_hybrid/gen_4dbar/all_dyn/'
if not os.path.exists(directory):
    os.makedirs(directory)
i = 0
##for root, dirs, files in os.walk('../musegan/exps/NowBarRNN_hybrid/gen_4dbar/all/', topdown=False):
for root, dirs, files in os.walk('../exps/nowbar_hybrid/gen_4dbar/all/', topdown=False):    
    for name in files:
        print(i)
        file = os.path.join(root, name)
        pm = pretty_midi.PrettyMIDI(file)
        print(np.shape(pm.instruments))
        offset = 30
        
        try:
            notes0 = pm.instruments[0].notes
            for note in notes0:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset +30 - a[0]
        except:
            pass
        
        try:
            notes1 = pm.instruments[1].notes
            for note in notes1:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset - a[0]
        except:
            pass
        
        try:
            notes2 = pm.instruments[2].notes
            for note in notes2:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset - a[0]
        except:
            pass
        
        try:
            notes3 = pm.instruments[3].notes
            for note in notes3:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset - a[0]
        except:
            pass
        
        try:
            notes4 = pm.instruments[4].notes
            for note in notes4:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset -10 - a[0]
        except:
            pass
        
        try:
            notes5 = pm.instruments[5].notes
            for note in notes5:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset - a[0]
        except:
            pass
        try:
            notes6 = pm.instruments[6].notes
            for note in notes6:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset - a[0]
        except:
            pass
        try:
            notes7 = pm.instruments[7].notes
            for note in notes7:
                a = np.random.randint(15, size=1)
                note.velocity = note.velocity -offset - a[0]
        except:
            pass
        ##pm.write('../musegan/exps/NowBarRNN_hybrid/gen_4dbar/all_dyn/'+name)
        pm.write('../exps/nowbar_hybrid/gen_4dbar/all_dyn/'+name)
        i += 1

0
(8,)
1
(8,)
2
(8,)
3
(8,)
4
(8,)
5
(8,)
6
(6,)
7
(7,)
8
(7,)
9
(8,)
10
(8,)
11
(8,)
12
(8,)
13
(6,)
14
(8,)
15
(8,)
16
(8,)
17
(8,)
18
(8,)
19
(8,)
20
(8,)
21
(7,)
22
(8,)
23
(8,)
24
(8,)
25
(8,)
26
(7,)
27
(8,)
28
(8,)
29
(8,)
30
(7,)
31
(7,)
32
(8,)
33
(8,)
34
(8,)
35
(7,)
36
(8,)
37
(8,)
38
(8,)
39
(8,)
40
(8,)
41
(8,)
42
(8,)
43
(8,)
44
(8,)
45
(8,)
46
(7,)
47
(8,)
48
(8,)
49
(8,)
50
(7,)
51
(6,)
52
(6,)
53
(7,)
54
(8,)
55
(8,)
56
(8,)
57
(8,)
58
(8,)
59
(8,)
60
(8,)
61
(7,)
62
(8,)
63
(8,)
64
(8,)
